In [794]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import copy
#print('copy: {}'.format(copy.__version__))

import math
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import word_tokenize


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
scipy: 1.3.0
numpy: 1.16.4
matplotlib: 3.1.0
pandas: 0.24.2
sklearn: 0.21.2


[nltk_data] Downloading package stopwords to /Users/noah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/noah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [795]:
import pandas
import collections
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from gensim.models import Word2Vec

In [796]:
dataset = pandas.read_csv('dataset/REF_PRODUIT_TAGS_train.csv',sep='|')


def strtolist(s):
    s = s.strip("[]")
    s = s.split(',')
    return s

def get_tags(dataset):
    dic = collections.defaultdict(int)
    for x in dataset['tags']:
        new_list = strtolist(x)
        for tag in new_list:
            dic[tag] += 1
    return dic
tags = get_tags(dataset)





In [797]:
def more_rec(dic,n):
    sorted_dic = sorted(dic.items(), key=lambda kv: kv[1],reverse=True) 
    return sorted_dic[:n], sorted_dic[n:]
tags_to_keep, tags_to_remove = more_rec(tags,300)
tags_to_keep = [i[0] for i in tags_to_keep]
tags_to_remove = [i[0] for i in tags_to_remove]

In [798]:
{dataset["SEG_REF_CAPA_TYPE"][i] for i in range(dataset.shape[0])}

{'KG', 'L', 'U', nan}

In [799]:
def sanitize_tags(dataset,tags_to_remove, tags_to_keep):
    new_dataset = dataset.copy()
    empty = 0
    for i in range(dataset.shape[0]):
        ith_element = new_dataset["tags"][i]
#         deleted =[]
        l = strtolist(ith_element)
        l1 = list(l)
        for x in l1:
            if x not in tags_to_keep:
                l.remove(x)
        if len(l) == 0:
#             deleted.append(ith_element)
#             new_dataset.drop(dataset.index[i])
#             print(new_dataset.shape[0])
            new_dataset.at[i,'tags'] = None
            empty += 1
        else:
            l = [tags_to_keep.index(x) for x in l]
            new_dataset.at[i,'tags'] = l
        
        seg = new_dataset["SEG_REF_CAPA_TYPE"][i]
        if seg not in ['KG', 'L', 'U']:
            new_dataset.at[i,'SEG_REF_CAPA_TYPE'] = 'Not described'
            
    
    return new_dataset.dropna(), empty#,deleted

new_dataset, empty = sanitize_tags(dataset, tags_to_remove, tags_to_keep)
new_dataset.dropna()
new_dataset= new_dataset.reset_index(drop = True)




In [800]:
new_dataset

,BARCODE,DESC1,DESC2,BRAND_DESC,Rayon,Famille,Sous_famille,SEG_REF_CAPA_TYPE,CAPA_VOLUME,SANS_SUCRE,PORC,SANS_GLUTEN,BIO,HALAL,CASHER,tags
0,7290006319470,PREPARATION SODA CONCENTRE CITRON VERT SODASTREAM,SIROP CONCENTRE CITRON VERT SODASTREAM CONC. B...,SODASTREAM,PEM,ACC.QUALITE ENVIRONNEMENT,ACC QUALITE ENVIRONNEMENT,U,1.000,0,0,0,0,0,0,"[2, 120, 12, 38]"
1,3033710074525,CAFE INSTANTANE NESCAFE,NESTLE NESCAFE SPEC FILT NATURELLEMENT CAFEINE...,NESCAFE,EPICERIE,CAFES SOLUBLES ET CHICOREE,CAFES SOLUBLES,KG,0.200,0,0,0,0,0,0,"[0, 2, 1, 37, 53, 206]"
2,3039820171253,POIS CASSES DE NOS AGRICULTEURS BIO VIVIEN PAILLE,SOUFFLET ALIMENTAIRE VIVIEN PAILLE POIS CASSE ...,VIVIEN PAILLE,EPICERIE,SEMOULES/COUSCOUS,BLE/AUTRES CEREALES,KG,0.500,0,0,0,0,0,0,"[0, 1, 64, 80, 78, 195, 214]"
3,3041090008269,"PLAT BEBE DES 12 MOIS, SPAGHETTI BOLOGNAISE BL...",PLAT BLEDICHEF SPAGHETTI BOLOGNAISE BLEDINA BL...,BLEDINA,EPICERIE,"REPAS LEGUME,LEGUM/VIANDE",ASSIETTE LEG VIANDE OU POISSON,KG,0.230,0,0,0,0,0,0,"[10, 43, 175]"
4,3041090717451,"PETITS POTS BEBE DES 6 MOIS, POMMES MANGUES BL...",PETIT POT FRUIT 6/36 MOIS POMME MANGUE BLEDINA...,BLEDINA,EPICERIE,"JUS FRUITS, FRUIT, GOUTER",FRUITS,KG,0.520,0,0,0,0,0,0,"[0, 1, 13, 11, 25, 43, 75, 136, 141, 294]"
5,3045470102110,BISCUITS BARQUETTES POMME-BANANE POULT,BISCUIT BARQUETTE POMME BANANE POULT BISCUIT P...,PP NO NAME,EPICERIE,"CHOCOLATES, FRUITES",CONFITURES,KG,0.120,0,0,0,0,0,0,"[3, 4, 14, 28]"
6,3083681003130,HARICOTS VERTS EXTRA-FINS VAPEUR BONDUELLE,BONDUELLE BONDUELLE CONS LEG HAR VT EXTRA FIN ...,BONDUELLE,EPICERIE,CONSERVES LEG.VERTS,HARICOTS VERTS,KG,0.440,0,0,0,0,0,0,"[0, 1, 64, 22, 11, 80, 55, 32, 76, 156]"
7,3088544504014,MIEL D'ACACIA LUNE DE MIEL,FAMILLE MICHAUD LUNE DE MIEL (FAMILLE MICHAUD)...,REFS.SANS MARQUE,EPICERIE,MIELS ET CREMES,MIELS,KG,0.450,0,0,0,0,0,0,"[17, 23, 40, 116, 70, 72, 121]"
8,3168930010739,CHIPS SPICY LAY'S,LAY'S CHIPS CHIPS SPICY FORMAT FAMILIAL 1 CT 2...,LAY'S,EPICERIE,CHIPS,CHIPS FORMAT FAMILIAL,KG,0.240,0,0,0,0,0,0,"[0, 1, 3, 7, 34, 41, 81, 102, 150]"
9,3173287630608,CHOCOLATS PAPILLOTES CROUSTILLANTES CEMOI,CHOCOLAT PAPILLOTE DROLEMENT CROUSTILLANTE SAI...,CEMOI,EPICERIE,CHOCOLATS SAISONNIERS NOEL,SPECIALITE,KG,0.380,0,0,0,0,0,0,"[3, 4, 27, 66, 91, 279, 85]"


In [801]:
with open("dict.txt",'r') as out:
    voc = []
    for sentance in out.readlines():
        sentance = sentance.strip()
        sentance = re.sub(r'\d+', '', sentance)
        sentance = sentance.replace('é','e')
        sentance = sentance.replace('û','u')
        sentance = re.sub(r'[^\w\s]','',sentance)
        voc.append(sentance)
print(voc)

['Succulent', 'Savoureux', 'Aromatique', 'Allege', 'Gouteux', 'Long en bouche', 'Surprenant', 'Original', 'Banal', 'Ordinaire', 'Fumant', 'Surgele', 'Fume', 'Collant', 'Pâteux', 'Craquant', 'Grumeleux', 'Gluant', 'Granuleux', 'Juteux', 'Croquant', 'Moelleux', 'Texture', 'Gout', 'Sec', 'Bon', 'Mauvais', 'Delicieux', 'Acre', 'Exquis', 'Fade', 'Insipide', 'Acerbe', 'Rance', 'Ageable', 'Aggressif', 'Dominant', 'Fort', 'Fermente', 'Gelatineux', 'Amer', 'Sucre', 'Acide', 'Astringent', 'Aigre', 'Doux', 'Sale', 'Cremeux', 'Epais', 'Onctueux', 'Cru', 'Cuit', 'Leger', 'Lourd', 'Fin', 'Saignant', 'A point', 'Ferme', 'Copieux', 'Compact', 'Dense', 'Mousseux', 'Aere', 'Acidule', 'Chaud', 'Froid', 'Tiède', 'Glace', 'Brulant', 'Bouillant', 'Affine', 'Liquide', 'Croustillant', 'Intense', 'Gras', 'Farineux', 'Serre', 'Poivre', 'Epice', 'Piquant', 'Releve', 'Fondant', 'Dur', 'Tendre', 'Grille', 'Veloute', 'Brule', 'Calcine', 'Frais', 'Pasteurise', 'Mou', 'Petillant', 'Dore']


In [802]:
from collections import Counter

stop = stopwords.words('french')
stop = [x.upper() for x in stop]
def most_common(dataset,entry):
    dic = collections.defaultdict(int)
    for x in dataset[entry]:
        sentance = x.replace(',',' ')
        sentance = sentance.replace('/',' ')
        sentance = re.sub(r'\d+', '', sentance)
        sentance = sentance.replace('\\',' ')
        sentance = re.sub(r'[^\w\s]',' ',sentance)
        for y in sentance.split():
            dic[y] += 1
    res = sorted(dic.keys(),key = lambda p : dic[p],reverse = True)
    
    for x in res:
        if len(x) <4 or x in stop:
            res.remove(x)
    return res

fam_common = most_common(new_dataset,'Famille')[:]
subfam_common = most_common(new_dataset,'Sous_famille')[:]
desc1_common = most_common(new_dataset,'DESC1')[:600]
desc2_common = most_common(new_dataset,'DESC2')[:600]
def sigmoid(x):
     return 1 / (1 + math.exp(-x))
def split_famille(x):
    sentance = x.replace(',',' ')
    sentance = sentance.replace('/',' ')
    sentance = re.sub(r'\d+', '', sentance)
    sentance = sentance.split()
    
    return sentance
def extract(dataset):    
    # declare result
    products = [None] * len(dataset['Rayon'])
    for i in range(len(products)):
        products[i] = []

    # add small info
#     ,'SEG_REF_CAPA_TYPE','SANS_SUCRE','PORC','SANS_GLUTEN'
    useful_data_info = ['Rayon']
    for info in useful_data_info:
        count = Counter(dataset[info])
        list_ = list(dict(count).keys())

        temp_count = 0
        for element in dataset[info]:
            temp = []
            
            for i in range(len(list_)):
                if list_[i] == element:
                    temp.append(sigmoid(i))
                    
            products[temp_count] = products[temp_count] + temp
            
            temp_count += 1
    
#     add famille
    for i in range(len(dataset['Famille'])):
        x = dataset['Famille'][i]
        sentance = split_famille(x)
        temp_vector = [0] * len(fam_common)
        for word in sentance:
            if word in fam_common:
                temp_vector[fam_common.index(word)] +=len(fam_common) - fam_common.index(word)
        products[i] = products[i] + temp_vector
    
    # add sous famille
    for i in range(len(dataset['Sous_famille'])):
        x = dataset['Sous_famille'][i]
        sentance = split_famille(x)
        temp_vector = [0] * len(subfam_common)
        for word in sentance:
            if word in subfam_common:
                temp_vector[subfam_common.index(word)] += len(subfam_common) - subfam_common.index(word)
            
        products[i] = products[i] + temp_vector
    
    for i in range(len(dataset['DESC1'])):
        x = dataset['DESC1'][i]
        sentance = split_famille(x)
        temp_vector = [0] * len(desc1_common)
        for word in sentance:
            if word in desc1_common:
                temp_vector[desc1_common.index(word)] += len(desc1_common) - desc1_common.index(word)
        products[i] = products[i] + temp_vector
    for i in range(len(dataset['DESC2'])):
        x = dataset['DESC2'][i]
        sentance = split_famille(x)
        temp_vector = [0] * len(desc2_common)
        for word in sentance:
            if word in desc2_common:
                temp_vector[desc2_common.index(word)] += len(desc2_common) - desc2_common.index(word)
        products[i] = products[i] + temp_vector

    
    return products

output = [x for x in new_dataset["tags"]]


features = extract(new_dataset)

In [803]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA

In [804]:
type(features)

list

In [805]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

new_output = MultiLabelBinarizer().fit_transform(output)




# forest = RandomForestClassifier(n_estimators=20, random_state=1)

# multi_target_forest = MultiOutputClassifier(forest,n_jobs = -1)
# # multi_target_forest = OneVsRestClassifier(LinearSVC(random_state=0))
# # multi_target_forest = sklearn.neural_network.MLPClassifier()
# scores = multi_target_forest.fit(features, new_output)
# result_train = scores.predict(features)


from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier


# clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
# scores = cross_val_score(clf, features, new_output, cv=5)
# scores.mean()                               


# clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
# scores = cross_val_score(clf, features, new_output, cv=5)
# scores.mean()                               


clf = ExtraTreesClassifier(n_estimators=20, max_depth=None,min_samples_split=2, random_state=0)
scores = clf.fit(features,new_output)
result_train = scores.predict(features)

# clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(15, 2), random_state=1)
# scores = clf.fit(features,new_output)
# result_train = scores.predict(features)



In [806]:
def res(trained,features):
    count = 0
    n = len(trained)
    
    for i in range(n):
        nb_gd_tags = 0
        nb_wr_tags = 0
        total = len(features[i])
        to_predict = 0 
#         features[i] = np.array(features[i])
        for j in range(len(features[i])):
            if trained[i][j] == 1 and features[i][j] == 0:
                    nb_wr_tags += 1
#             elif trained[i][j] == 0 and features[i][j] == 1:
#                     nb_wr_tags += 1
                    
            elif trained[i][j] == 1 and features[i][j] == 1:
                    nb_gd_tags += 1
            if features[i][j] == 1:
                to_predict += 1
                
        count += nb_gd_tags /(nb_wr_tags+to_predict)
        
    return count/n *100

print(res(result_train,new_output))

99.32229736998156


In [807]:
dataset_test = pandas.read_csv('dataset/REF_PRODUIT_TAGS_test.csv',sep='|')

new_dataset_test, empty = sanitize_tags(dataset_test, tags_to_remove, tags_to_keep)
new_dataset_test.dropna()
new_dataset_test =new_dataset_test.reset_index(drop=True)
new_dataset_test= new_dataset_test.reset_index(drop = True)
features_test = extract(new_dataset_test)
output_test = [x for x in new_dataset_test["tags"]]


new_output_test = MultiLabelBinarizer().fit_transform(output_test)

result_test = scores.predict(features_test)


print(len(result_test))
print(len(new_output_test))

print(res(result_test,new_output_test))


4278
4278
76.61901770246797
